In [1]:
#python packages

#Bert Model
from transformers import pipeline, AutoTokenizer, BertForMaskedLM
unmasker = pipeline('fill-mask', model='dumitrescustefan/bert-base-romanian-cased-v1', top_k=10)

#tokenizer
import spacy
NLP = spacy.load("ro_core_news_lg")

#text classification
import fasttext.util
fasttext.util.download_model('ro', if_exists='ignore')
ft = fasttext.load_model('cc.ro.300.bin')

#zipf_score
from wordfreq import zipf_frequency

#cosine similarity
import scipy.spatial, scipy.special

#min max scaler
from sklearn.preprocessing import MinMaxScaler

C:\Users\Anghel Fabian\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
import json

synonyms_dict = {}
antonyms_dict = {}

#load synonyms and antonyms
#with open(join(dirname(__file__), 'synonyms.json'), 'r') as f:
with open('scraping/synonyms.json', 'r') as f:
    synonyms_dict = json.load(f)

#with open(join(dirname(__file__), 'antonyms.json'), 'r') as f:
with open('scraping/antonyms.json', 'r') as f:
    antonyms_dict = json.load(f)

In [3]:
def preprocess_sentence(sentence):
    #get right versions of ș/ț
    sentence = sentence.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

    #transform a/i + unicode 770/u+0302 in â/î
    sentence = sentence.replace("a\u0302", "â").replace("i\u0302", "î").replace("A\u0302", "Â").replace("I\u0302", "Î")

    #transform s/t + unicode 807/u+0327 or 806/u+0326 in ș/ț
    sentence = sentence.replace("s\u0327", "ș").replace("t\u0327", "ț").replace("S\u0327", "Ș").replace("T\u0327", "Ț")
    sentence = sentence.replace("s\u0326", "ș").replace("t\u0326", "ț").replace("S\u0326", "Ș").replace("T\u0326", "Ț")

    #transform a + unicode 774/u+0306 in ă
    sentence = sentence.replace("a\u0306", "ă").replace("A\u0307", "Ă")

    return sentence


In [4]:
def valid_token(token, zipf_score):
    #if token is punctuation
    if token.is_punct:
        return False
    
    #if token is number
    if token.is_digit or token.like_num:
        return False
    
    #if token is whitespace
    if token.is_space:
        return False
    
    #if token is currency:
    if token.is_currency:
        return False
    
    #if token is url or email
    if token.like_url or token.like_email:
        return False
    
    #too high zipf score
    if zipf_score > 4.25:
    #if zipf_score > 4.5:
    #if zipf_score > 4:
        return False
    
    #to do: check ENTs
    #some can be general words (like "îmbrăcăminte", "litoral", "alcool")
    #DATETIME should still be replaced
    #check if it is proper noun
    
    return True

In [5]:
def word_candidates(sentence, new_tokens):
    tokens = NLP(sentence)
    replacable_tokens = []

    #sort tokens by Zipf score
    for token in tokens:
        #token_score = zipf_frequency(token.text, 'ro')
        token_score = zipf_frequency(token.lemma_, "ro")
        if valid_token(token, token_score) and token.text not in new_tokens:
            #context = extract_context(tokens, token.i)
            replacable_tokens.append((token, token_score))

    replacable_tokens.sort(key=lambda x: x[1])

    return replacable_tokens

In [6]:
def substitution_generation(sentence, token):
    # generate a masked sentence in this form: [CLS] original sentence [SEP] masked sentence [SEP]
    masked_sentence = sentence[:token.idx] + "[MASK]" + sentence[token.idx + len(token.text):]
    model_sentence = "[CLS] " + sentence + " [SEP] " + masked_sentence + " [SEP]"
    result = unmasker(model_sentence)
    suggestions = []

    for x in result:
        #check if suggestion is same part of speech 
        replaced_sentence = sentence[:token.idx] + x["token_str"] + sentence[token.idx + len(token.text):]
        replaced_tokens = NLP(replaced_sentence)

        if token.pos != replaced_tokens[token.i].pos and not (token.pos_ == "AUX" and replaced_tokens[token.i].pos_ == "VERB") and not (token.pos_ == "VERB" and replaced_tokens[token.i].pos_ == "AUX"):
            continue

        #check if it is the same word or same lemma
        #if token.lemma_ == replaced_tokens[token.i].lemma_ or token.text == replaced_tokens[token.i].text:
            #continue

        #check if suggestion is not negative version of word
        #if "ne" + token.text == x["token_str"] or "ne" + x["token_str"] == token.text:
            #continue

        suggestions.append(x)

    return suggestions    

In [7]:
def synonym_score(suggestion, token):
    sugg_token = NLP(suggestion)[0]
    sugg_lemma = sugg_token.lemma_
    token_lemma = token.lemma_

    #positive score if synonyms
    if (token_lemma in synonyms_dict and sugg_lemma in synonyms_dict[token_lemma]) or (sugg_lemma in synonyms_dict and token_lemma in synonyms_dict[sugg_lemma]):
        return 1

    #negative score if antonyms
    if (token_lemma in antonyms_dict and sugg_lemma in antonyms_dict[token_lemma]) or (sugg_lemma in antonyms_dict and token_lemma in antonyms_dict[sugg_lemma]):
        return 0
    if "ne" + token_lemma == sugg_lemma or "ne" + sugg_lemma == token_lemma:
        return 0
    if "in" + token_lemma == sugg_lemma or "in" + sugg_lemma == token_lemma:
        return 0
    
    #negative score if same word to encourage different suggestions
    if token_lemma == sugg_lemma:
        return 0.25
    
    #netural score if not related
    return 0.5

In [ ]:
def substitution_ranking(suggetions, token):
    substitutions = []

    bert_scores = []
    zipf_scores = []
    fasttext_scores = []
    synonym_scores = []

    # calculate a score for each suggestion based on BERT, Zipf and FastText
    for x in suggetions:
        bert_scores.append([x["score"]])

        zipf_score = zipf_frequency(x["token_str"], 'ro')
        zipf_scores.append([zipf_score])

        fasttext_score = 1 - scipy.spatial.distance.cosine(ft[token.text], ft[x["token_str"]])
        fasttext_scores.append([fasttext_score])

        syn_score = synonym_score(x["token_str"], token)
        synonym_scores.append([syn_score])
    
    if len(bert_scores) > 0:
        bert_scaler = MinMaxScaler()
        zipf_scaler = MinMaxScaler()
        fasttext_scaler = MinMaxScaler()
        synonym_scaler = MinMaxScaler()

        bert_scores_norm = bert_scaler.fit_transform(bert_scores)
        zipf_scores_norm = zipf_scaler.fit_transform(zipf_scores)
        fasttext_scores_norm = fasttext_scaler.fit_transform(fasttext_scores)
        synonym_scores_norm = synonym_scaler.fit_transform(synonym_scores)

        for i in range(len(suggetions)):
            final_score = (bert_scores_norm[i] + zipf_scores_norm[i] + fasttext_scores_norm[i] + synonym_scores[i]) / 4
            substitutions.append((suggetions[i]["token_str"], final_score))

        # sort suggestions by score and replace the word with the best suggestion if it has a higher Zipf score
        substitutions.sort(reverse=True, key=lambda x: x[1])
        #print(substitutions)

    return substitutions

In [ ]:
def sentence_simplification(sentence):
    token_replaced = True
    new_tokens = set([])

    #preprocessing
    sentence = preprocess_sentence(sentence)

    #sent_tokens = NLP(sentence)
    #for token in sent_tokens:
        #if token.ent_iob != 2:
            #print(token.text, token.ent_type_)
        #print(token.text, token.pos_, token.tag_)

    while token_replaced:
        token_replaced = False

        replacable_tokens = word_candidates(sentence, new_tokens)
        
        for token in replacable_tokens:
            if token_replaced:
                break

            token_text = token[0].text

            suggestions = substitution_generation(sentence, token[0])
            suggestions = substitution_ranking(suggestions, token[0])

            if len(suggestions) == 0:
                continue
            
            top_replacement = suggestions[0][0]

            if zipf_frequency(top_replacement, 'ro') >= zipf_frequency(token_text, 'ro'):
                sentence = sentence[:token[0].idx] + top_replacement + sentence[token[0].idx + len(token_text):]
                new_tokens.add(top_replacement)
                token_replaced = True

    return sentence

In [10]:
def text_simplification(text):
    doc = NLP(text)
    for sentence in doc.sents:
        new_sentence = sentence_simplification(sentence.text)
        text = text.replace(sentence.text, new_sentence)
    return text

In [11]:
#normal sentences
sentence_list = ["Priveliștea era spectaculoasă și se întindea până la orizont.", 
                 "Ei consumaseră prea mult alcool în acea seară.", 
                 "Mihai a mers la mall ca să își achiziționeze îmbrăcăminte.",
                 "Eu și gașca mea am mers în vacanță la litoral.",
                 "Am mâncat prea multă ciocolată și am făcut hiperglicemie.", 
                 "La facultate am studiat despre algoritmi și structuri de date.",
                 "Articolul publicat de mine a fost citit de toată lumea.",]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Priveliștea NOUN Ncfsry
era AUX Vaii3s
spectaculoasă ADJ Afpfsrn
și CCONJ Crssp
se PRON Px3--a--------w
întindea AUX Vmii3s
până ADP Spsa
la ADP Spsa
orizont NOUN Ncms-n
. PUNCT PERIOD
[('spectaculoasă', array([0.56925676])), ('frumoasă', array([0.50722348])), ('impresionantă', array([0.43570325])), ('minunată', array([0.32771924])), ('superbă', array([0.30635682])), ('extraordinară', array([0.29316899])), ('uimitoare', array([0.28322702])), ('incredibilă', array([0.27725993])), ('încântătoare', array([0.15099732])), ('uluitoare', array([0.1445889]))]
[('Marea', array([0.5730695])), ('Pădurea', array([0.52185274])), ('Apa', array([0.45234795])), ('Noaptea', array([0.38898478])), ('priveliștea', array([0.33254303])), ('Natura', array([0.30862809])), ('Plaja', array([0.2596716])), ('Cascada', array([0.19530147]))]
[('orizont', array([0.775])), ('mare', array([0.37719151])), ('cer', array([0.35555292])), ('pământ', array([0.26440883])), ('soare', array([0.25163591])), ('apus', array([0.21

In [34]:
#complex sentences
sentence_list = ["Procrastinarea constantă în fața responsabilităților academice poate duce la o exacerbare a anxietății și la compromiterea performanțelor intelectuale pe termen lung.",
                 "Implementarea unor strategii pedagogice inovatoare poate facilita internalizarea cunoștințelor teoretice într-un mod eficient și sustenabil.",
                 "Perturbarea echilibrului ecologic cauzată de defrișările necontrolate generează consecințe ireversibile asupra biodiversității și climatului global.",
                 "Diseminarea informațiilor eronate prin canale digitale poate conduce la dezinformare masivă și la erodarea încrederii publice în instituțiile fundamentale.",
                 "Complexitatea proceselor administrative influențează negativ eficiența deciziilor guvernamentale.",
                 "Polarizarea ideologică din societate poate compromite coeziunea comunitară și exacerba conflictele latente."]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

[('Creșterea', array([0.57796192])), ('Expunerea', array([0.52580022])), ('Pierderea', array([0.48923982])), ('Frica', array([0.48549577])), ('Poziția', array([0.46590735])), ('Durerea', array([0.46581211])), ('Atitudinea', array([0.4542556])), ('Implicarea', array([0.44375639])), ('Deschiderea', array([0.42834273])), ('Greutatea', array([0.21520619]))]
[('responsabilităților', array([0.5625])), ('responsabilității', array([0.47263795])), ('responsabilitățile', array([0.45013738])), ('obligațiilor', array([0.44219394])), ('sarcinilor', array([0.4393758])), ('atribuțiilor', array([0.40105923])), ('competențelor', array([0.39762815])), ('riscurilor', array([0.38818303])), ('obiectivelor', array([0.37661841])), ('limitelor', array([0.3478892]))]
[('creștere', array([0.74540576])), ('dezvoltare', array([0.49447904])), ('scădere', array([0.42128332])), ('ameliorare', array([0.39618983])), ('diminuare', array([0.37691797])), ('extindere', array([0.36440619])), ('reducere', array([0.31359962]

In [35]:
#special sentences
sentence_list = ["Au trecut peste 147 de ani de la înființarea Universității din București, iar eu am studiat acolo trei ani.", 
                 "Găsiți mai multe informații pe https://spacy.io/api/token.",]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

147 DATETIME
de DATETIME
ani DATETIME
Universității ORGANIZATION
din ORGANIZATION
București ORGANIZATION
trei DATETIME
ani DATETIME
[('înființarea', array([0.76640071])), ('crearea', array([0.47932801])), ('apariția', array([0.46388851])), ('nașterea', array([0.37587471])), ('constituirea', array([0.3670231])), ('deschiderea', array([0.3631898])), ('desființarea', array([0.33236587])), ('fondarea', array([0.27381636])), ('întemeierea', array([0.24132007])), ('absolvirea', array([0.17343372]))]
[('studiat', array([0.58423913])), ('fost', array([0.40555082])), ('lucrat', array([0.38885273])), ('stat', array([0.38617769])), ('învățat', array([0.32072556])), ('rămas', array([0.27916484])), ('trăit', array([0.25574332])), ('petrecut', array([0.24372992])), ('predat', array([0.24257107])), ('locuit', array([0.20612218]))]
Au trecut peste 147 de ani de la înființarea Universității din București, iar eu am studiat acolo trei ani.
Găsiți mai multe informații pe https://spacy.io/api/token.


In [36]:
text = "În contextul actual al dezvoltării societății contemporane, se observă o multitudine de problematici emergente care influențează semnificativ calitatea vieții cotidiene. Diversitatea factorilor determinanți contribuie la creșterea complexității soluțiilor propuse pentru ameliorarea acestor deficiențe. De exemplu, în domeniul educației, implementarea unor strategii pedagogice elaborate și utilizarea unor metodologii avansate au devenit priorități esențiale. Totuși, o parte considerabilă a populației rămâne marginalizată din cauza lipsei accesibilității resurselor educaționale. Pe de altă parte, în sectorul economic, dinamica pieței și globalizarea contribuie la apariția unor provocări substanțiale pentru întreprinderile mici și mijlocii. În acest sens, este imperativ să fie identificate modalități eficiente pentru stimularea sustenabilității acestor organizații. Un alt aspect relevant este cel legat de sănătate. Progresul tehnologic a facilitat crearea unor dispozitive medicale inovative, însă accesul la acestea este, din păcate, limitat pentru o parte semnificativă a populației."

print(text_simplification(text))

[('emergente', array([0.5625])), ('majore', array([0.38107146])), ('viitoare', array([0.37543326])), ('existente', array([0.36729299])), ('actuale', array([0.35297004])), ('fundamentale', array([0.34190002])), ('contemporane', array([0.33051005])), ('globale', array([0.32354342]))]
[('probleme', array([0.82399357])), ('aspecte', array([0.42704503])), ('teme', array([0.42361192])), ('întrebări', array([0.41598694])), ('chestiuni', array([0.40375891])), ('subiecte', array([0.39945208])), ('domenii', array([0.39491275])), ('situații', array([0.28573394])), ('provocări', array([0.27572046])), ('fenomene', array([0.20251856]))]
[('multitudine', array([0.70454545])), ('serie', array([0.46660836])), ('varietate', array([0.42280678])), ('mulțime', array([0.41218275])), ('gamă', array([0.33691205])), ('diversitate', array([0.29167669])), ('combinație', array([0.27727273])), ('sumedenie', array([0.26222192])), ('suită', array([0.20941447])), ('paletă', array([0.18183228]))]
[('cotidiene', array(